In [23]:
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

import mlflow
import mlflow.h2o
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

import pandas as pd
import json

from sklearn.metrics import f1_score, accuracy_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

In [24]:
# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 day 2 hours 19 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,29 days
H2O_cluster_name:,tamilarasan
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.298 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [28]:
# Initialize MLFlow client
client = MlflowClient()

# Set up MlFlow experiment
experiment_name = 'automl-insurance-classification'

try:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = client.get_experiment_by_name(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)
    
mlflow.set_experiment(experiment_name)

# Print experiment details
print(f"Name: {experiment_name}")
print(f"Experiment_id: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")
print(f"Tags: {experiment.tags}")
print(f"Lifecycle_stage: {experiment.lifecycle_stage}")
print(f"Tracking uri: {mlflow.get_tracking_uri()}")

Name: automl-insurance-classification
Experiment_id: 1
Artifact Location: /home/tamilarasan/projects/lectures/mlflow/artifacts/1
Tags: {}
Lifecycle_stage: active
Tracking uri: http://localhost:5000


In [29]:
# Import data directly as H2O frame
main_frame = h2o.import_file(path='data/processed/train.csv')

# Save data types of columns in H2O frame (for matching with test set during prediction)
with open('data/processed/train_col_types.json', 'w') as fp:
    json.dump(main_frame.types, fp)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [30]:
# Set predictor and target columns
target = 'Response'
predictors = [n for n in main_frame.col_names if n != target]

# Factorize target variable so that autoML tackles classification problem (instead of regression)
main_frame[target] = main_frame[target].asfactor()

# Visualize H2O frame structure
main_frame.head()

Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_4,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,Product_Info_2_Alpha,Product_Info_2_Num,Med_Keywords_Count
1,14,26,0.487179,2,1,1,0.164179,0.818182,0.435146,0.576961,0.027,9,1,0,2,0.15,1,2,1,2,1,1,1,2,1,3,2,0.000133333,1,3,2,3,0.376812,0.253521,22,491,2,2,1,3,2,2,1,3,2,3,3,1,3,1,1,2,1,2,3,1,3,3,1,3,2,3,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3,1,0
1,5,26,0.230769,2,3,1,0.268657,0.781818,0.368201,0.517129,0.1,12,1,0,2,0.35,2,2,6,3,1,1,1,2,1,1,3,nan,3,2,3,2,nan,0.394366,nan,162,2,2,1,3,2,2,2,3,2,3,3,1,3,1,1,2,1,2,3,1,3,3,1,3,2,3,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,6,0
1,0,26,1,2,3,1,0.507463,0.654545,0.299163,0.545946,0.15,12,1,0,2,1,2,2,3,3,1,1,1,2,1,3,1,0.00233333,1,3,2,3,nan,nan,1,261,2,2,1,3,2,2,2,3,2,3,3,3,3,1,1,2,1,2,3,2,2,3,1,3,2,3,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,1,1
1,16,26,0.230769,2,3,1,0.134328,0.763636,0.215481,0.296359,0.042,9,1,0,2,nan,2,2,8,3,1,1,1,1,1,3,1,0.000666667,1,1,2,3,nan,nan,1,407,2,2,1,3,2,2,2,3,2,3,3,1,3,1,1,2,1,2,3,1,3,3,1,3,2,3,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,3,3,0
1,17,26,0.487179,2,3,1,0.492537,0.618182,0.276151,0.546823,0.12,9,1,0,2,0.12,1,2,3,3,1,1,1,1,1,3,1,0.001,1,1,2,3,nan,0.450704,0,132,2,1,1,3,2,2,2,3,2,3,3,1,3,1,1,2,1,2,3,2,2,3,1,1,2,3,3,3,1,3,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,3,4,2
1,16,26,1,2,3,1,0.58209,0.654545,0.278243,0.506623,0.115,9,1,0,2,1,1,2,8,3,1,2,1,1,1,3,2,0.00472,1,1,2,3,nan,0.661972,8,261,2,1,1,3,2,2,1,3,2,3,3,1,3,1,1,2,2,2,3,1,3,3,1,1,2,3,3,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
mlflow.set_tracking_uri("http://localhost:5000")

# Wrap autoML training with MLflow
with mlflow.start_run(log_system_metrics=True):
    aml = H2OAutoML(
                    max_models=2, # Run AutoML for n base models
                    seed=42, 
                    balance_classes=True, # Our target classes are imbalanced, so we set this to True
                    sort_metric='logloss', # Sort models by logloss (main metric for multi-classification)
                    verbosity='info', # Turn on verbose info
                    exclude_algos = ['GBM','GLM', 'DRF','StackedEnsemble','DeepLearning'], # Specify which algorithms to exclude
                   )
    
    aml.train(x=predictors, y=target, training_frame=main_frame)
    
    # Set metrics to log
    mlflow.log_metric("log_loss", aml.leader.logloss())
    mlflow.log_metric("AUC", aml.leader.auc())
    
    # Log best model (mlflow.h2o module provides API for logging & loading H2O models)
    mlflow.h2o.log_model(aml.leader, 
                         artifact_path="model",
                         registered_model_name="h20-model"
                        )
    
    model_uri = mlflow.get_artifact_uri("model")
    print(model_uri)
    
    # Print and view AutoML Leaderboard
    lb = get_leaderboard(aml, extra_columns='ALL')
    print(lb.head(rows=lb.nrows))
    
    # Get IDs of current experiment run
    exp_id = experiment.experiment_id
    run_id = mlflow.active_run().info.run_id
    
    # Save leaderboard as CSV
    lb_path = f'mlruns/{exp_id}/{run_id}/artifacts/model/leaderboard.csv'
    lb.as_data_frame().to_csv(lb_path, index=False) 
    print(f'Leaderboard saved in {lb_path}')

2024/06/11 16:47:13 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/06/11 16:47:13 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


AutoML progress: |


16:47:13.223: Project: AutoML_23_20240611_164713
16:47:13.223: 5-fold cross-validation will be used.
16:47:13.223: Setting stopping tolerance adaptively based on the training frame: 0.004104535616689248
16:47:13.223: Build control seed: 42
16:47:13.223: training frame: Frame key: AutoML_23_20240611_164713_training_py_9_sid_9d7c    cols: 124    rows: 59357  chunks: 32    size: 6954768  checksum: -1088187092662504757
16:47:13.223: validation frame: NULL
16:47:13.223: leaderboard frame: NULL
16:47:13.223: blending frame: NULL
16:47:13.223: response column: Response
16:47:13.223: fold column: null
16:47:13.223: weights column: null
16:47:13.224: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}, {DeepLearning

2024/06/11 16:55:28 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/11 16:55:28 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


model_id                               logloss    mean_per_class_error      rmse       mse    training_time_ms    predict_time_per_row_ms  algo
XGBoost_2_AutoML_23_20240611_164713    1.27888                0.624656  0.653082  0.426517               34437                   0.010921  XGBoost
XGBoost_1_AutoML_23_20240611_164713    1.29889                0.626317  0.653736  0.427371               52110                   0.013557  XGBoost
[2 rows x 8 columns]



OSError: Cannot save file into a non-existent directory: 'mlruns/1/e17e0dff88d643979443093f9e3547f6/artifacts/model'

In [32]:
# Get AutoML event log
log = aml.event_log
log

timestamp,level,stage,message,name,value
16:36:03.221,INFO,Workflow,Project: AutoML_22_20240611_163603,,
16:36:03.221,INFO,Validation,5-fold cross-validation will be used.,,
16:36:03.222,INFO,Validation,Setting stopping tolerance adaptively based on the training frame: 0.004104535616689248,,
16:36:03.222,INFO,Validation,Build control seed: 42,,
16:36:03.222,INFO,DataImport,training frame: Frame key: AutoML_22_20240611_163603_training_py_9_sid_9d7c cols: 124 rows: 59357 chunks: 32 size: 6954768 checksum: -1088187092662504757,,
16:36:03.222,INFO,DataImport,validation frame: NULL,,
16:36:03.222,INFO,DataImport,leaderboard frame: NULL,,
16:36:03.222,INFO,DataImport,blending frame: NULL,,
16:36:03.222,INFO,DataImport,response column: Response,,
16:36:03.222,INFO,DataImport,fold column: null,,


In [18]:
# Leader (best) model stored here
aml.leader

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_2_AutoML_21_20240611_154734


Model Summary: 
    number_of_trees
--  -----------------
    43

ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.29341086722519744
RMSE: 0.5416741337974312
LogLoss: 0.8366127543973001
Mean Per-Class Error: 0.3548035456937989
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1     2     3    4    5     6      7     8      Error     Rate
----  ----  ---  ---  ----  -----  ----  -----  --------  ---------------
3550  280   9    39   229   757    365   978    0.428065  2,657 / 6,207
221   3825  18   39   285   856    374   934    0.416209  2,727 / 6,552
21    32    576  23   70    196    31    64     0.431392  437 / 1,013
20    13    3    741  0     316    51    284    0.481092  687 / 1,428
96    140   8    6    3896  605    247   434    0.282769  1,536 / 5,432
198   151   3    58   193   7936   828   1866   0.29351   3,297 / 11,233
103   50    5    30   35    993    4455  2356   0.444998  3,572 / 8,027
55    37    1    50   27    620    387   18312  0.060393  1,177 / 19,489
4264  4528  623  986  4735  12279  6738  25228  0.270962  16,090 / 59,381

Top-8 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.729038
2    0.891565
3    0.952746
4    0.982115
5    0.993584
6    0.998299
7    0.99973
8    1

ModelMetricsMultinomial: xgboost
** Reported on cross-validation data. **

MSE: 0.42610793548468934
RMSE: 0.6527694351642771
LogLoss: 1.2818227205611092
Mean Per-Class Error: 0.623599590679947
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1     2     3    4    5     6      7     8      Error     Rate
----  ----  ---  ---  ----  -----  ----  -----  --------  ---------------
1645  965   32   53   458   1263   564   1227   0.734977  4,562 / 6,207
919   1739  37   71   700   1362   548   1176   0.734585  4,813 / 6,552
84    92    93   66   231   324    35    88     0.908193  920 / 1,013
48    45    13   236  2     523    74    487    0.834734  1,192 / 1,428
332   558   77   12   2645  1056   283   469    0.513071  2,787 / 5,432
678   560   36   141  510   5488   1403  2417   0.51144   5,745 / 11,233
243   205   6    56   56    1609   3008  2844   0.625265  5,019 / 8,027
165   134   2    140  42    1189   794   17023  0.126533  2,466 / 19,489
4114  4298  296  775  4644  12814  6709  25731  0.463178  27,504 / 59,381

Top-8 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.536822
2    0.732238
3    0.84746
4    0.918071
5    0.960644
6    0.984204
7    0.993988
8    1

Cross-Validation Metrics Summary: 
                         mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
-----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
accuracy                 0.536822  0.0045314   0.533889      0.539491      0.538986      0.530397      0.541344
aic                      nan       0           nan           nan           nan           nan           nan
auc                      nan       0           nan           nan           nan           nan           nan
err                      0.463178  0.0045314   0.466111      0.460509      0.461014      0.469603      0.458656
err_count                5500.8    53.8906    

In [22]:
# Import test data
test_frame = h2o.import_file(path='data/processed/test.csv')
test_frame.head()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
1,1,D3,26,0.487179,2,3,1,0.61194,0.781818,0.338912,0.472262,0.15,3,1,0,2,0.5,2,2,11,3,1,1,1,2,1,1,3,nan,3,2,3,3,nan,0.627451,0.760563,nan,2,16,2,2,1,3,1,2,2,nan,3,2,1,3,nan,1,2,1,1,2,1,2,1,nan,2,2,1,1,3,2,3,nan,3,3,1,3,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,A2,26,0.0769231,2,3,1,0.626866,0.727273,0.311715,0.484984,0,1,3,0.07,2,0.2,1,2,8,3,1,1,1,1,1,3,1,0.00166667,1,1,2,2,nan,0.529412,0.746479,nan,5,261,3,1,1,3,2,2,1,nan,3,2,3,3,110,3,3,1,1,2,1,2,3,nan,2,2,3,1,3,2,3,nan,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,D3,26,0.144667,2,3,1,0.58209,0.709091,0.320084,0.519103,0.143,9,1,0,2,0.45,1,2,3,3,1,1,1,2,1,1,3,nan,3,2,3,3,0.666667,nan,0.661972,nan,3,132,2,1,1,3,2,2,2,nan,3,2,3,3,240,1,3,1,1,2,1,2,3,nan,2,2,3,1,1,2,3,nan,1,3,1,3,2,1,3,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,A1,26,0.151709,2,1,1,0.522388,0.654545,0.267782,0.486962,0.21,9,1,0,2,1,2,2,3,3,1,1,1,1,1,3,1,0.000666667,2,1,2,2,nan,0.686275,0.676056,nan,nan,162,3,2,1,1,2,3,2,nan,3,2,3,3,nan,1,3,1,1,2,2,2,3,nan,1,3,3,2,3,2,3,nan,3,1,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
12,1,A1,26,0.0769231,2,3,1,0.298507,0.672727,0.246862,0.428718,0.085,9,1,0,2,0.2,1,2,8,3,1,2,1,2,1,1,3,nan,3,2,3,2,0.449275,nan,0.380282,nan,18,181,3,1,1,3,2,2,2,nan,3,2,3,3,188,1,3,1,1,2,1,2,1,nan,1,3,3,1,1,2,3,nan,3,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1,D3,26,0.230769,2,3,1,0.567164,0.818182,0.299163,0.379754,0.075,9,1,0,2,0.4,1,2,8,3,1,1,1,2,1,1,3,nan,3,2,3,3,n

In [ ]:
# Drop ID column for test set
X_test_frame = test_frame.drop('Response')
y_test_frame = test_frame[:, 'Response']

In [6]:
# Get dataframe of all runs
all_experiments = [exp.experiment_id for exp in client.list_experiments()]
runs = mlflow.search_runs(experiment_ids=all_experiments, run_view_type=ViewType.ALL)

# Identify best model (experiment id and run id) amongst all runs in the experiment
run_id, exp_id = runs.loc[runs['metrics.log_loss'].idxmin()]['run_id'], runs.loc[runs['metrics.log_loss'].idxmin()]['experiment_id']
run_id, exp_id

NameError: name 'client' is not defined

In [7]:
# Load best model (AutoML leader)
best_model = mlflow.h2o.load_model(f"mlruns/{exp_id}/{run_id}/artifacts/model/")

# Generate predictions with best model (output is H2O frame)
preds_frame = best_model.predict(X_test_frame)

NameError: name 'mlflow' is not defined

In [8]:
# Get y values (ground truth and predicted)
y_pred = preds_frame.as_data_frame()['predict']
y_true = y_test_frame.as_data_frame()['Response']

NameError: name 'preds_frame' is not defined

In [9]:
from sklearn.metrics import f1_score, accuracy_score
f1_score(y_true, y_pred)

NameError: name 'y_true' is not defined

In [10]:
accuracy_score(y_true, y_pred)

NameError: name 'y_true' is not defined